<a href="https://colab.research.google.com/github/julija-dmrk/data-mining/blob/main/4th_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

classification

In [18]:
import matplotlib.pyplot as plt   # visualization
import seaborn as sns             # visualization
import numpy as np                # data manipulation
import pandas as pd               # data manipulation, data processing, CSV file I/O (e.g. pd.read_csv
from sklearn import datasets, metrics, pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut, cross_val_score, cross_validate, GridSearchCV, train_test_split
from sklearn.model_selection import cross_val_score, cross_validate 

In [19]:
url = "https://raw.githubusercontent.com/VitaT/ML-python/main/data/suicides.csv"
data = pd.read_csv(url)

In [20]:
# cleaning up
# spliting country-year column
data['year'] = data["country-year"].str[-4:]
data['country'] = data["country-year"].str.split("\d+", expand = True)[0]
# removing ID and country-year columns
data.drop(["ID", "country-year"], axis=1, inplace=True)

In [21]:
# renaming columns for more convenient use
data.rename(columns={'gdp_for_year ($)':'gdp_year', 'gdp_per_capita ($)':'gdp_capita', 'suicides/100k pop':'suicide_norm', "HDI for year":"hdi_year"}, inplace=True)

In [22]:
data.isna().sum()
# suicide per 100k people can be calculated from suicides_no and population. Let's check it they are the same
# We can overwrite suicide_norm
data["suicide_norm"] = round(data["suicides_no"] / data["population"] * 100000,  2)

In [49]:
#Classification
features = data.drop(["suicides_no", "suicide_norm", "hdi_year"], axis = 1)
target = np.array(data["suicides_no"])

features = pd.get_dummies(features, columns=["sex", "age", "year", "generation", "country"])

#features = data.data
#target = data.target

In [50]:
# Data preparatation. 
# Create standardizer and standardize features
features_standardized = StandardScaler().fit_transform(features)
# use train and test split
# multiple classification, gr = 3
x, x_out, y, y_out = train_test_split(features_standardized, target, test_size = 0.1, random_state = 0)  # for multiple category clasification
# binary classification, gr = 2
x2, x2_out, y2, y2_out = train_test_split(features_standardized[:100,], target[:100], test_size = 0.1, random_state = 0)  # for binary clasification

In [52]:
# Create logistic regression object
log_regr = LogisticRegression(random_state=0)
# Train model
cv_log_reg = cross_validate(log_regr, x2, y2, cv=10, error_score="accuracy", return_train_score=True)

for key in ["test_score", "train_score"]:
  print(key)
  print("Accuracy: %0.2f (+/- %0.2f)" % (cv_log_reg.get(key).mean(), cv_log_reg.get(key).std() * 2))
# 2 SD is roughly 95 % of data.


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown

test_score
Accuracy: 0.24 (+/- 0.22)
train_score
Accuracy: 0.86 (+/- 0.07)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
